In [1]:
# 03 Autoregression

In [2]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
weatherclimateED = pd.read_csv('weatherclimateED.csv', parse_dates = [0], dayfirst = True)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import itertools
import statsmodels.api as sm
from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')
from IPython.display import clear_output
import os
from joblib import Parallel, delayed

In [3]:
## Epiweeks Module converts dates to CDC Epiweek format
## Further documentation on https://pypi.org/project/epiweeks/
from epiweeks import Week, Year
from datetime import date
def create_epiweek(date):
    return Week.fromdate(date)
def create_epiweekplot(epiweek):
    epiweek = str(epiweek)
    return F'Y{epiweek[:4]}W{epiweek[4:]}'
def create_epiweek_fromstr(str):
    return Week.fromstring(str)

In [4]:
## This section creates a full complete dataset that includes all the variables of interest that will be used
## iloc function selects the relevant variables of interest based on column number
## Problematic weather columns (i.e. don't select!): 6, 16, 17, 19, 20
## Disease columns excluded due to limited dataset: 21:24, 25

weatherclimateED['epiweek'] = weatherclimateED['Date'].apply(create_epiweek)
weatherclimateED = weatherclimateED.set_index('epiweek')
weatherclimateED = weatherclimateED.iloc[:, np.r_[30:32, 33:39, 40, 42 , 45:47, 49:51,  52:54, 1:6, 8:15]]
weatherclimateED.info()

<class 'pandas.core.frame.DataFrame'>
Index: 866 entries, 200601 to 202231
Data columns (total 28 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Cardiovascular disease                                              679 non-null    float64
 1   Chronic respiratory disease                                         679 non-null    float64
 2   Diabetes mellitus                                                   679 non-null    float64
 3   Digestive disease                                                   679 non-null    float64
 4   Endocrine disorders                                                 679 non-null    float64
 5   Factors influencing health status and contact with health services  679 non-null    float64
 6   Genitourinary disorders                                             679 non-null    float64
 7   Ill-defined di

In [5]:

## This function takes the full dataset and creates an initial dataset with the specified range
## also returns the name of the target variable for creation of the initial dataset
## note disease_var here is an integer based off the column number
def create_initial_dataset(dataset, disease_var: int):
    explore_df = dataset.copy()
    range_start = Week(2009,1)
    range_end = Week (2018,52)
    explore_df = explore_df.loc[range_start:range_end]
    target_var = explore_df.columns.values.tolist()[disease_var]

    if not os.path.exists(target_var):
        os.makedirs(target_var)
    path = os.path.join(target_var, F'initial_dataset.csv')
    
    explore_df.to_csv(path)
    #explore_df.info()
    #explore_df

    return explore_df, target_var

In [6]:
def create_naive(dataset, target_var):
    naive = dataset.copy()
    naive = naive[[target_var]].shift(1)
    return naive.dropna()

In [7]:
# Create lagged dataset
def create_lagged_dataset(dataset, lag, target_var):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna()

In [8]:
## Step is the number of weeks ahead that we are forecasting, e.g. step=2 is 2 weeks ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var):
    stepped_dataset = dataset.copy()
    y = stepped_dataset[[target_var]].shift(-step+1)
    if step != 1:
        X = stepped_dataset.iloc[:-step+1, :]
    else:
        X = stepped_dataset
    return X.drop(target_var, axis = 1), y.dropna()
## So now target variable (y variable for exploration) is shifted back by 2 weeks. i.e., taking the y-value from 2 weeks later
## and setting it to the current index. So linear regression of y+2 with the current X values. X will have
## a smaller dataset with the last 2 time points removed because of the shift. 

In [9]:
def create_window(X, window_perc):
    return X.index[0], X.index[int(len(X)*window_perc)]
def create_output_dataset(y, window_end):
    return y.copy().loc[window_end+1:]

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

def errors(model, model_name, X_train, y_train, errors_path, filename):
    if model_name == 'naive':
        y_train_errors = pd.DataFrame(X_train - y_train)
    else:
        y_train_errors = pd.DataFrame(model.predict(X_train) - y_train)
    #print(y_train_errors)
    #y_train_errors will contain list of errors in training set from window_start to window_end
    #right before window_end+1 in the filename
    errors_path = os.path.join(errors_path, model_name)
    if not os.path.exists(errors_path):
        os.makedirs(errors_path)
    y_train_errors_path = os.path.join(errors_path, F'{filename}.csv')
    y_train_errors.to_csv(y_train_errors_path)

def coefs(model, coefs_path, filename):
    coefs_path

## This function runs the first order regression for the target disease, for one specified lag and step

def regression_with_naive(X_dataset, y_dataset, window_start, window_end, y_pred, y_params, errors_path, test_length, naive, target_var, y_ridge, y_lasso):
    count = 0
    df_end = X_dataset.index[-1]
    while window_end != df_end:
        X = X_dataset.copy()
        y = y_dataset.copy()
        # Note: .loc is end-inclusive    
        X_train = X.loc[window_start:window_end]
        #print(X_train.info())
        ## values.ravel() converts y_train to numpy array for compatibility with models
        y_train = y.loc[window_start:window_end].values.ravel()
        #print(len(y_train))
        ## double square brackets so X_test is extracted as a pandas df instead of series
        X_test = X.loc[[window_end+1]]
        #print(X_test)
        y_test = y.loc[window_end+1]
        #print(y_test)
    
        ## Scaling
        scaler = StandardScaler()
        ## .fit_transform stores the scaling parameters (fit), and transforms the training set
        X_train = scaler.fit_transform(X_train)
        ## .transform takes the previously stored scaling parameters to transform the test set
        ## Therefore, test set is transformed based on the training set parameters
        X_test = scaler.transform(X_test)
    
        ## evaluate variance
        '''
    
        ## Naive Forecast
        # add the [0] to extract as float, and not as series
        y_pred.at[window_end+1, 'naive'] = naive.loc[window_end+1][0]

        errors(naive, 'naive', naive.loc[window_start:window_end].values.ravel(), y_train, errors_path, window_end+1)
        
        
        ## Linear Regression Model
        linreg_model = LinearRegression()
        # Fit the model to the training data
        linreg_model.fit(X_train, y_train)
        # Make predictions and store
        y_pred.at[window_end+1, 'linreg'] = linreg_model.predict(X_test)

        errors(linreg_model, 'linreg', X_train, y_train, errors_path, window_end+1)
        '''
    
        ## Implement cross-validation split
        tscv = TimeSeriesSplit(n_splits = 5)
    
        ## Ridge model
        ridge_cv = RidgeCV(cv = tscv)
        ridge_cv.fit(X_train, y_train)
    
        ridge_model = Ridge(alpha = ridge_cv.alpha_)
        ridge_model.fit(X_train, y_train)
        
        #y_pred.at[window_end+1, 'ridge'] = ridge_model.predict(X_test)
        #y_params.at[window_end+1, 'ridge_alpha'] = ridge_cv.alpha_

        alpha = ridge_cv.alpha_
        ridge_edf = 0
        for d in ridge_model.coef_:
            ridge_edf += d**2/(d**2+alpha)
        
        y_ridge.at[window_end+1, 'ridge_edf'] = ridge_edf
        
        #errors(ridge_model, 'ridge', X_train, y_train, errors_path, window_end+1)

        
        ## Lasso Model
        lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
        lasso_cv.fit(X_train, y_train)
        
        # Create the Lasso model with the optimal alpha value
        lasso_model = Lasso(alpha = lasso_cv.alpha_)
        lasso_model.fit(X_train, y_train)
        #y_pred.at[window_end+1, 'lasso'] = lasso_model.predict(X_test)
        #y_params.at[window_end+1, 'lasso_alpha'] = lasso_cv.alpha_
        #y_params.at[window_end+1, 'lasso_n_iter'] = lasso_cv.n_iter_
        
        y_lasso.at[window_end+1, 'lasso_edf'] = np.count_nonzero(lasso_model.coef_)

        #errors(lasso_model, 'lasso', X_train, y_train, errors_path, window_end+1)
        
        
        '''
        ## ElasticNet Model
        elasticnet_cv = ElasticNetCV(cv = tscv, max_iter = 100000)
        elasticnet_cv.fit(X_train, y_train)
    
        # Create the ElasticNet model with the optimal l1 and alpha values
        elasticnet_model = ElasticNet(alpha = elasticnet_cv.alpha_, l1_ratio = elasticnet_cv.l1_ratio_)
        elasticnet_model.fit(X_train, y_train)
        y_pred.at[window_end+1, 'elasticnet'] = elasticnet_model.predict(X_test)
        
        y_params.at[window_end+1, 'elasticnet_alpha'] = elasticnet_cv.alpha_
        y_params.at[window_end+1, 'elasticnet_l1ratio'] = elasticnet_cv.l1_ratio_

        errors(elasticnet_model, 'elasticnet', X_train, y_train, errors_path, window_end+1)
        
        ## Random Forest
        randomforest_model = RandomForestRegressor(n_estimators = 1000, max_features = 'sqrt', random_state = 18)
        randomforest_model.fit(X_train, y_train)
        y_pred.at[window_end+1, 'randomforest'] = randomforest_model.predict(X_test)
        errors(randomforest_model, 'randomforest', X_train, y_train, errors_path, window_end+1)
    
        ## Gradient Boost
        gradientboost_model = GradientBoostingRegressor(n_estimators = 1000, random_state = 18)
        gradientboost_model.fit(X_train, y_train)
        y_pred.at[window_end+1, 'gradientboost'] = gradientboost_model.predict(X_test)
        errors(gradientboost_model, 'gradientboost', X_train, y_train, errors_path, window_end+1)
    
        ## KNN
        knn_parameters = {'n_neighbors': range(1, 40), 'weights': ['uniform', 'distance']}
        #round(len(y_train)*0.15-1))
        knn_cv = GridSearchCV(KNeighborsRegressor(), knn_parameters, cv = tscv)
        knn_cv.fit(X_train, y_train)
        #knn_model = knn_cv.predict(X_test)
        #knn_model = KNeighborsRegressor()
        #knn_model.fit(X_train, y_train)
        y_pred.at[window_end+1, 'knn'] = knn_cv.predict(X_test)
        y_params.at[window_end+1, 'knn_n'] = knn_cv.best_estimator_
        errors(knn_cv, 'knn', X_train, y_train, errors_path, window_end+1)
        '''
        ##
        #keep track of model progress, every number of weeks
        tracking_interval = 5
        if window_end.weektuple()[1] % tracking_interval == 0:
            print(F'{target_var} done with {window_end+1}; {count} out of {test_length}')
            
        ## Implement expanding window
        #window_start = window_start+1 (only for rolling window)
        window_end += 1
        count += 1

    print(F'The last epiweek for {target_var} to be predicted is: {window_end}')
    print(F'The total number of predicted epiweeks for {target_var} is: {count}')


In [11]:
## This function sets up the first order regression for the target disease, for one specified lag and step

def run_first_order_regression(dataset, lag, step, target_var, window_perc):
    print(F'Running first order regression for {target_var} lag {lag} step {step}')
    
    naive = create_naive(dataset, target_var)
    #print(naive.info())
    
    lagged_dataset = create_lagged_dataset(dataset, lag, target_var)
    #print(lagged_dataset)
    
    X, y = create_stepped_dataset(lagged_dataset, step, target_var)
    #print(X.info())
    #print(y)
    
    window_start, window_end = create_window(X, window_perc)

    print(F'The first epiweek to be predicted for {target_var} lag {lag} step {step} is: {window_end+1}')
    
    y_pred = create_output_dataset(y, window_end)
    y_params = create_output_dataset(y, window_end)
    y_ridge = create_output_dataset(y, window_end)
    y_lasso = create_output_dataset(y, window_end)

    train_length = len(X.loc[window_start:window_end])
    print(F'The initial training dataset length for {target_var} lag {lag} step {step} is: {train_length}')


    test_length = len(X.loc[window_end+1:])
    print(F'The initial testing dataset length for {target_var} lag {lag} step {step} is: {test_length}')

    errors_path = os.path.join(target_var, 'errors', F'L{lag}_S{step}')
    
    if not os.path.exists(errors_path):
        os.makedirs(errors_path)
        
    regression_with_naive(X, y, window_start, window_end, y_pred, y_params, errors_path, test_length, naive, target_var, y_ridge, y_lasso)

    #pred_path = os.path.join(target_var, 'pred')
    #params_path = os.path.join(target_var, 'params')
    
    ridge_path = os.path.join(target_var, 'ridge_param')
    lasso_path = os.path.join(target_var, 'lasso_param')

    '''
    if not os.path.exists(pred_path):
        os.makedirs(pred_path)
    if not os.path.exists(params_path):
        os.makedirs(params_path)
    '''
    if not os.path.exists(ridge_path):
        os.makedirs(ridge_path)
    if not os.path.exists(lasso_path):
        os.makedirs(lasso_path)

    #pred_path = os.path.join(pred_path, F'L{lag}_S{step}.csv')
    #params_path = os.path.join(params_path, F'L{lag}_S{step}.csv')
    ridge_path = os.path.join(ridge_path, F'L{lag}_S{step}.csv')
    lasso_path = os.path.join(lasso_path, F'L{lag}_S{step}.csv')    


    
    #y_pred.to_csv(pred_path)
    #y_params.to_csv(params_path)

    y_ridge.to_csv(ridge_path)
    y_lasso.to_csv(lasso_path)
    

    print(F'Completed for {target_var} lag {lag} step {step}')
    clear_output(wait=False)
    return y_ridge, y_lasso

In [12]:
## This function runs the regression for one disease, for all lags and steps, hence the for loop

def run_disease_regression(dataset, disease_var, lag_start, lag_end, step_start, step_end):
    
    ## Note how the integer disease_var is input into this function, and then
    ## the string target_var is returned for the remaining functions
    explore_df, target_var = create_initial_dataset(dataset, disease_var)

    with open("target_variables.txt") as target_variables_file:
        if target_var not in target_variables_file.read():
            with open("target_variables.txt", 'a') as target_variables_file:
                target_variables_file.write(F'{target_var}\n')
    
    ## run the first order regression for all lags and steps for this target variable
    print(F'Running regression for {target_var}')
    for lag in range(lag_start, lag_end):
        for step in range(step_start, step_end):
            run_first_order_regression(explore_df, lag = lag, step = step, target_var = target_var, window_perc = 0.7)

In [13]:
## Deprecated as now using the Parallel and run_disease_regression functions
'''
def run_full_regression(dataset, disease_list, lag_start, lag_end, step_start, step_end):
    for disease_var in disease_list:
        run_disease_regression(dataset, disease_var, lag_start, lag_end, step_start, step_end)
'''

In [14]:
## Main function call using Parallel
## x in range (0,16) represents the 16 diseases that are the target variables. However, for this function we input them as integers
## the create_initial_dataset function will convert the integer format to string format
## Using parallel, each disease can be run on one computer core
Parallel(n_jobs=-2, verbose=51)(delayed(run_disease_regression)(weatherclimateED, x, 8, 9, 1, 13) for x in range(0,16))
#run_full_regression(weatherclimateED, range(0,16), 8, 9, 1, 9)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed: 38.0min
[Parallel(n_jobs=-2)]: Done   2 out of  16 | elapsed: 41.2min remaining: 288.2min
[Parallel(n_jobs=-2)]: Done   3 out of  16 | elapsed: 214.1min remaining: 927.9min
[Parallel(n_jobs=-2)]: Done   4 out of  16 | elapsed: 218.0min remaining: 654.0min
[Parallel(n_jobs=-2)]: Done   5 out of  16 | elapsed: 232.7min remaining: 511.8min
[Parallel(n_jobs=-2)]: Done   6 out of  16 | elapsed: 253.2min remaining: 421.9min
[Parallel(n_jobs=-2)]: Done   7 out of  16 | elapsed: 260.9min remaining: 335.4min
[Parallel(n_jobs=-2)]: Done   8 out of  16 | elapsed: 557.8min remaining: 557.8min
[Parallel(n_jobs=-2)]: Done   9 out of  16 | elapsed: 576.1min remaining: 448.1min
[Parallel(n_jobs=-2)]: Done  10 out of  16 | elapsed: 582.8min remaining: 349.7min
[Parallel(n_jobs=-2)]: Done  11 out of  16 | elapsed: 610.8min remaining: 277.6min
[Parallel(n_jobs=-2)]: Done  12 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]